In [1]:
from IPython.display import display, HTML
import scipy.constants as sc
from scipy.optimize import fsolve
from ipywidgets import interact, widgets
%matplotlib notebook

import numpy as np
import cmath
import matplotlib.pyplot as plt
import scipy.constants as sc
plt.rcParams.update({
    "text.usetex": True,
    'text.latex.preamble': r"\usepackage{amsmath} \boldmath"
})
import os
import h5py

print(os.getpid())
%cd ../

15617
/media/work/docs/codes/QuCF/scripts-py


In [2]:
# --- Launch to change cells' width ---
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pylib.mix as mix
import pylib.qucf_read as qucf_r
import pylib.qucf_oracle as qucf_o
path_ = "../simulations/tests/test-simple-matrix"
file_name_oracle_ = "circuit_OH"

In [4]:
# ---------------------------------------------------------------
# --- Read the matrix ---
# ---------------------------------------------------------------
mix.reload_module(qucf_r)

dd3 = qucf_r.read_matrix_sparse(path_, "test_n3_OUTPUT.hdf5") 
# dd4 = qucf_r.read_matrix(path_, "test_n4_OUTPUT.hdf5") 
# dd5 = qucf_r.read_matrix(path_, "test_n5_OUTPUT.hdf5") 
# dd6 = qucf_r.read_matrix(path_, "test_n6_OUTPUT.hdf5") 
# dd7 = qucf_r.read_matrix(path_, "test_n7_OUTPUT.hdf5") 
# dd8 = qucf_r.read_matrix(path_, "test_n8_OUTPUT.hdf5") 

Reading the matrix from: test_n3_OUTPUT.hdf5
from the path: ../simulations/tests/test-simple-matrix
date of the simulation:  08-12-2023 13:45:02
matrix name:  U_BE
N = 8


In [11]:
# --------------------------------------------------------------
# --- Chose the case ---
# --------------------------------------------------------------
mix.reload_module(qucf_r)
mix.reload_module(mix)
mix.reload_module(qucf_o)

from termcolor import colored


dd = dict(dd3)

# Form the matrix:
N = dd["N"]
A = np.zeros((N,N), dtype=complex)


a = 1.2
# b = 10.4
# f = 8.2
# c = 4.6

# A[0, 1] = b*1j;    A[0, 2] = a
# A[1, 0] = b*1j;    A[1, 2] = f;         A[1, 3] = 0.40
# A[2, 0] = a;    A[2, 1] = f;         A[2, 3] = c + f*1j; A[2, 4] = 0.3
# A[3, 1] = 0.40; A[3, 2] = c + f*1j;  A[3, 4] = c + f*1j; A[3, 5] = 0.3
# for i in range(4, N-4):
#     A[i, i-2] = 0.3; A[i, i-1] = c + f*1j; A[i, i+1] = c + f*1j;   A[i,i+2] = 0.3
# kk = N-4
# A[kk, kk-2] = 0.3; A[kk, kk-1] = c + f*1j; A[kk, kk+1] = c + f*1j; A[kk, kk+2] = 0.4
# kk = N-3
# A[kk, kk-2] = 0.3; A[kk, kk-1] = c + f*1j; A[kk, kk+1] = 0.15;     A[kk, kk+2] = 0.3
# kk = N-2
# A[kk, kk-2] = 0.4; A[kk, kk-1] = 0.15;       A[kk, kk+1] = 0.10; 
# kk = N-1
# A[kk, kk-2] = 0.3; A[kk, kk-1] = 0.10;    




# A[0, 1] = a;    A[0, 2] = a
# A[1, 0] = 0;    A[1, 2] = a;  A[1, 3] = a
# A[2, 0] = a;    A[2, 1] = a;  A[2, 3] = a; A[2, 4] = a
# A[3, 1] = a;    A[3, 2] = 0;  A[3, 4] = a; A[3, 5] = a
# A[4, 2] = a;    A[4, 3] = a;  A[4, 5] = 0; A[4, 6] = a
# A[5, 3] = a;    A[5, 4] = a;  A[5, 6] = a; A[5, 7] = 0
# A[6, 4] = 0;    A[6, 5] = 0;  A[6, 7] = a;
# A[7, 5] = a;    A[7, 6] = a;  

A[0, 1] = a;    A[0, 2] = a
A[1, 0] = a;    A[1, 2] = a;  A[1, 3] = a
A[2, 0] = a;    A[2, 1] = a;  A[2, 3] = a; A[2, 4] = a
A[3, 1] = a;    A[3, 2] = a;  A[3, 4] = a; A[3, 5] = a
A[4, 2] = a;    A[4, 3] = a;  A[4, 5] = a; A[4, 6] = a
A[5, 3] = a;    A[5, 4] = a;  A[5, 6] = a; A[5, 7] = a
A[6, 4] = a;    A[6, 5] = a;  A[6, 7] = a;
A[7, 5] = a;    A[7, 6] = a; 

A = mix.form_sparse_matrix(A)


# Define the circuit structure (problem-specific part):
oo_circ = qucf_o.Circuit__()
anc = qucf_o.Regs__()
inp = qucf_o.Regs__()
anc.add_reg("ar", dd["regs"]["ar"], True)
inp.add_reg("r",  dd["regs"]["r"], False)
oo_circ.set_regs(inp, anc)
oo_circ.compute_N_registers()

# Check whether D and A matrices have the same number of nonzero elements:
D = dd["A"]

if D.get_Nnz() != A.get_Nnz():
    line_warning = "<<< WARNING: Nnz of D and A are different.>>>\n"
    line_warning += "<<< The oracle constructor does not work in this case. >>>"
    line_warning = colored(line_warning, 'red', attrs=['reverse', 'blink'])
    print()
    print(line_warning)
    print()
    
# The first normalization normalization:
D_values = D.get_values()
A_values = A.get_values()
coef_norm_D = np.min(np.min(np.abs(D_values[np.nonzero(D_values)])))
coef_norm_A = np.max(np.sqrt(np.sum(np.abs(A_values)**2)))

A_norm = A.copy()
values_norm = A_norm.get_values()
values_norm *= coef_norm_D 
if coef_norm_A > 1:
    values_norm /= coef_norm_A
print("normalized matrix >>>")
A_norm.print_max_min()
del D_values, A_values, coef_norm_D, coef_norm_A, values_norm

# The second normalization:
B_fixed = A_norm.copy()
B_values = B_fixed.get_values()
for ii in range(B_fixed.get_Nnz()):
    B_values[ii] /= D.v(ii)
print("N-nonzero in the flat-elements-matrix: ", int(B_fixed.get_Nnz()))
del B_values

normalized matrix >>>
amax. value: 			4.903e-02
amin.(excl. zero) value: 	4.903e-02

N-nonzero in the flat-elements-matrix:  26


In [27]:
# -----------------------------------------------
# --- Computation of the oracle's angles ---
# -----------------------------------------------
mix.reload_module(mix)
mix.reload_module(qucf_o)

# Rearrange angles:
grid_sections = qucf_o.create_grid_of_sections(oo_circ, B_fixed)

# -- Create sections of sets (sets = groups):
sys_gates_ = qucf_o.SystemGates__(
    oo_circ, 
#     qucf_o.create_groups(oo_circ, grid_sections)        # Option 1
    qucf_o.create_groups_neighbor(oo_circ, grid_sections) # Option 2
)
print("--- Initial number of groups ---")
sys_gates_.count_groups()


# Recreate the matrix using the ORIGINAL groups
print("--- Reconstruction using original groups ---")
sys_gates_.reconstruct_matrix_using_GROUPS().is_the_same_as(B_fixed, prec = oo_circ.prec_)

# # Split groups (optional):
# sys_gates_.n_split_ = 1
# sys_gates_.split_groups()
# sys_gates_.split_groups()
# sys_gates_.split_groups()
# sys_gates_.split_groups()
# sys_gates_.split_groups()
# print("--- After the splitting ---")
# sys_gates_.count_groups()

# Sort groups:
sys_gates_.sort_groups()

# Extend the gates:
sys_gates_.set_rows_limit_for_non_extension(2)
sys_gates_.extend_sorted_groups(B_fixed)

# merge groups:
sys_gates_.merge_groups()
print("\n--- After the merging ---")
sys_gates_.count_groups()

# preliminary correction (optional):
sys_gates_.correct_close_groups()

# add correcting groups:
sys_gates_.correct_groups(B_fixed)
print("--- After adding the correcting groups ---")
sys_gates_.count_groups()

# Recreate the matrix using the CORRECTED groups
print("--- Reconstruction using corrected groups ---")
sys_gates_.reconstruct_matrix_using_GRID().is_the_same_as(B_fixed, prec = oo_circ.prec_)

# Construct the circuit:
print("\n--- Circuit construction ---")
print("N-nonzero in the flat-elements-matrix: ", int(B_fixed.get_Nnz()))
sys_gates_.construct_circuit_OH(path_, file_name_oracle_)

--- Initial number of groups ---
Number of groups: 14

--- Reconstruction using original groups ---
The matrices are the same

--- After the merging ---
Number of groups: 14

--- After adding the correcting groups ---
Number of groups: 20

--- Reconstruction using corrected groups ---
The matrices are the same

--- Circuit construction ---
N-nonzero in the flat-elements-matrix:  26
N-gates: 26
N-gates (assuming Rc as a single gate): 20


In [28]:
# ----------------------------------------------------------
# --- Read results ---
# ----------------------------------------------------------
mix.reload_module(qucf_r)

dd_res = qucf_r.read_matrix_sparse(path_, "simple-matrix_OUTPUT.hdf5") 

dd_res["A"].is_the_same_as(A_norm, prec = oo_circ.prec_)

Reading the matrix from: simple-matrix_OUTPUT.hdf5
from the path: ../simulations/tests/test-simple-matrix
date of the simulation:  08-12-2023 14:26:16
matrix name:  U_BE
N = 8
The matrices are the same


In [29]:
# ---------------------------------------------------------------
# --- Function to print matrix ---
# ---------------------------------------------------------------
mix.reload_module(mix)

print("\n--- A ---")
A.print_matrix_real(
    0, 0, N,
    ff=[6, 3, "e"], 
    n_in_row = N, 
    gap_be = " ", 
)

print("\n--- A-norm ---")
print()
A_norm.print_matrix_real(
    0, 0, N,
    ff=[6, 3, "e"], 
    n_in_row = N, 
    gap_be = " ", 
)

print("\n--- D ---")
print()
D.print_matrix_real(
    0, 0, N,
    ff=[6, 3, "e"], 
    n_in_row = N, 
    gap_be = " ", 
)

print("\n--- B-fixed ---")
B_fixed.print_matrix_real(
    0, 0, N,
    ff=[6, 3, "e"], 
    n_in_row = N, 
    gap_be = " ", 
)

print("\n--- A-norm-Reconstructed ---")
dd_res["A"].print_matrix_real(
    0, 0, N,
    ff=[6, 3, "e"], 
    n_in_row = N, 
    gap_be = " ", 
)


--- A ---
--- 0:  0.000e+00 1.200e+00 1.200e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
--- 1:  1.200e+00 0.000e+00 1.200e+00 1.200e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
--- 2:  1.200e+00 1.200e+00 0.000e+00 1.200e+00 1.200e+00 0.000e+00 0.000e+00 0.000e+00
--- 3:  0.000e+00 1.200e+00 1.200e+00 0.000e+00 1.200e+00 1.200e+00 0.000e+00 0.000e+00
--- 4:  0.000e+00 0.000e+00 1.200e+00 1.200e+00 0.000e+00 1.200e+00 1.200e+00 0.000e+00
--- 5:  0.000e+00 0.000e+00 0.000e+00 1.200e+00 1.200e+00 0.000e+00 1.200e+00 1.200e+00
--- 6:  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.200e+00 1.200e+00 0.000e+00 1.200e+00
--- 7:  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.200e+00 1.200e+00 0.000e+00

--- A-norm ---

--- 0:  0.000e+00 4.903e-02 4.903e-02 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
--- 1:  4.903e-02 0.000e+00 4.903e-02 4.903e-02 0.000e+00 0.000e+00 0.000e+00 0.000e+00
--- 2:  4.903e-02 4.903e-02 0.000e+00 4.903e-02 4.903e-02 0.000e+00 0.000e+00 0.000e+00
--- 